# S-Q-L or Sequel?

- ¿Cómo creo una BBDD SQL?
- ¿Cómo añado/elimino/modifico datos?
- Triggers, ¿un tigre?
- ¿Por qué no debes olvidar el WHERE cuando uses DELETE FROM?
- Referencias y next steps

---------------------------------------------------------------------------------------------

In [1]:
import sqlite3
import pandas as pd

conn = sqlite3.connect('thebridge.db') 
# Se crea la base de datos si no encuentra una con ese nombre

In [2]:
cursor = conn.cursor() 
# La bbdd se guarda allí donde tengas tu archivo jupyter

### CREATE TABLE

Tened en cuenta que la query para crear nuevas tablas solo podrás ejecutarla una vez, sino dará un error porque ya está creada.  

La columna `[generated_id]` se usa para ir generando un ID incremental para cada entrada. 

Cuando se crea una tabla nueva en la misma query se puede añadir el nombre y el formato de las columnas o campos.


In [3]:
# Crear tabla - STUDENTS
cursor.execute('''CREATE TABLE STUDENTS
             ([generated_id] INTEGER PRIMARY KEY,
             [Student_Name] text, [Bootcamp_ID] integer, 
             [Date_Start] date)''')
          
# Crear tabla - BOOTCAMP
cursor.execute('''CREATE TABLE BOOTCAMP
             ([generated_id] INTEGER PRIMARY KEY,
             [Bootcamp_ID] integer, [Program] text)''')
        
# Crear tabla - STUDENT_LOGS - Tabla donde se registra cuando se registra un alumno nuevo
cursor.execute('''CREATE TABLE STUDENT_LOGS
             ([Student_ID] integer, [Date_Start] text)''')
                 
conn.commit()

Acerca del método `commit()` --> https://dev.mysql.com/doc/connector-python/en/connector-python-api-mysqlconnection-commit.html

In [4]:
query_all = "SELECT name FROM sqlite_master WHERE type='table';"
cursor.execute(query_all)
lista_tablas = cursor.fetchall()
print(lista_tablas)

[('STUDENTS',), ('BOOTCAMP',), ('STUDENT_LOGS',)]


### ALTER TABLE + ADD COLUMN

In [5]:
query_addColumn = "ALTER TABLE BOOTCAMP ADD COLUMN Bootcamp_Name text"

cursor.execute(query_addColumn)

### INSERT INTO 

In [6]:
cursor.execute("""
                INSERT INTO BOOTCAMP (Bootcamp_ID, Bootcamp_Name, Program)
                VALUES
                (11, 'Data Science', 'Full Time'),
                (21, 'Full Stack', 'Full Time'),
                (31, 'UX/UI', 'Full Time'),
                (41, 'Ciberseguridad', 'Full Time'),
                (12, 'Data Science', 'Part Time'),
                (22, 'Full Stack', 'Part Time'),
                (32, 'UX/UI', 'Part Time'),
                (42, 'Ciberseguridad', 'Part Time')
                """)

In [7]:
query_bootcamp = """
SELECT * FROM BOOTCAMP
"""
pd.read_sql_query(query_bootcamp, conn)

,generated_id,Bootcamp_ID,Program,Bootcamp_Name
0,1,11,Full Time,Data Science
1,2,21,Full Time,Full Stack
2,3,31,Full Time,UX/UI
3,4,41,Full Time,Ciberseguridad
4,5,12,Part Time,Data Science
5,6,22,Part Time,Full Stack
6,7,32,Part Time,UX/UI
7,8,42,Part Time,Ciberseguridad


In [10]:
cursor.execute('''
                INSERT INTO STUDENTS (Student_Name, Bootcamp_ID, Date_Start)
                VALUES
                ('Javi', 11, '20-11-2020'),
                ('Javi', 11, '25-01-2021'),
                ('Laura', 32, '20-11-2020'),
                ('Martín', 21, '25-01-2021'),
                ('Elena', 12, '25-01-2021')
                ''')


Si ejecuto esta celda dos veces, los valores se van a duplicar generando un ID único para cada valor.

In [9]:
cursor.execute('SELECT * FROM STUDENTS')

for row in cursor:
    print(row)

(1, 'Javi', 11, '20-11-2020')
(2, 'Javi', 11, '25-01-2021')
(3, 'Laura', 32, '20-11-2020')
(4, 'Martín', 21, '25-01-2021')
(5, 'Elena', 12, '25-01-2021')


In [11]:
query_students = """
SELECT * FROM STUDENTS
"""
pd.read_sql_query(query_students, conn)

,generated_id,Student_Name,Bootcamp_ID,Date_Start
0,1,Javi,11,20-11-2020
1,2,Javi,11,25-01-2021
2,3,Laura,32,20-11-2020
3,4,Martín,21,25-01-2021
4,5,Elena,12,25-01-2021
5,6,Javi,11,20-11-2020
6,7,Javi,11,25-01-2021
7,8,Laura,32,20-11-2020
8,9,Martín,21,25-01-2021
9,10,Elena,12,25-01-2021


### UPDATE

In [12]:
# Elena se quiere cambiar del Part Time de UX/UI al Full Time de Data Science
cursor.execute('''
                UPDATE STUDENTS
                SET Bootcamp_ID = 11
                WHERE Student_Name = 'Elena'
                ''')

pd.read_sql_query(query_students, conn)

,generated_id,Student_Name,Bootcamp_ID,Date_Start
0,1,Javi,11,20-11-2020
1,2,Javi,11,25-01-2021
2,3,Laura,32,20-11-2020
3,4,Martín,21,25-01-2021
4,5,Elena,11,25-01-2021
5,6,Javi,11,20-11-2020
6,7,Javi,11,25-01-2021
7,8,Laura,32,20-11-2020
8,9,Martín,21,25-01-2021
9,10,Elena,11,25-01-2021


### DELETE

In [13]:
cursor.execute('''
                DELETE FROM STUDENTS 
                WHERE generated_id in (8,9,10)
               ''')

pd.read_sql_query(query_students, conn)

,generated_id,Student_Name,Bootcamp_ID,Date_Start
0,1,Javi,11,20-11-2020
1,2,Javi,11,25-01-2021
2,3,Laura,32,20-11-2020
3,4,Martín,21,25-01-2021
4,5,Elena,11,25-01-2021
5,6,Javi,11,20-11-2020
6,7,Javi,11,25-01-2021


In [14]:
cursor.execute('''
                DELETE FROM STUDENTS 
                WHERE generated_id BETWEEN 6 AND 7
               ''')

pd.read_sql_query(query_students, conn)

,generated_id,Student_Name,Bootcamp_ID,Date_Start
0,1,Javi,11,20-11-2020
1,2,Javi,11,25-01-2021
2,3,Laura,32,20-11-2020
3,4,Martín,21,25-01-2021
4,5,Elena,11,25-01-2021


¡No te olvides de `conn.commit()` para hacer permanente tus cambios! 

Observa la importancia de la instrucción **WHERE**, en ella indicamos los registros que queremos borrar: todos los registros que cumplan con la condición indicada (es decir en cuya columna indicada esté el valor indicado) serán eliminados.

De no poner la instrucción **WHERE** se eliminarán todos los registros de la tabla. La tabla seguirá existiendo, pero será una tabla vacía, sin ningún registro (sólo se mantiene la estructura).

Hay que tener mucho cuidado con la instrucción **DELETE**, ya que la eliminación de un registro se hace de forma irreversible. Una vez eliminado ya no se pueden recuperar los datos.





In [15]:
conn.commit()

[Video "No te olvides de poner el WHERE en el DELETE FROM"](https://www.youtube.com/watch?v=i_cVJgIz_Cs&t=1s)

### TRIGGERS

Son objetos de la base de datos que ejecutan acciones cuando se producen ciertos eventos (inserciones, modificaciones, borrados, creación de tablas, etc).

Más sobre los triggers --> https://elbauldelprogramador.com/plsql-disparadores-o-triggers/

In [16]:
cursor.execute("drop trigger if exists my_trigger")

cursor.execute("""CREATE TRIGGER my_trigger
             AFTER INSERT ON STUDENTS
             BEGIN
                 INSERT INTO STUDENT_LOGS (Student_ID, Date_Start) 
                 VALUES (new.generated_id, strftime('%Y/%m-%H:%M', 'now'));
             END
             ;
             """)
# que no os pase lo que a mí, el error es que estaba poniendo %y (minúscula) y eso no es nada
# '2012-11-14 14:32:30' --> '%Y-%m-%d %H:%M:%S'

In [17]:
cursor.execute("""
                INSERT INTO STUDENTS (Student_Name, Bootcamp_ID, Date_Start)
                VALUES
                ('Clara', 42, '12-04-2021'),
                ('Borja', 21, '12-04-2021')
                """)

In [18]:
conn.commit()

In [19]:
lista_tablas

[('STUDENTS',), ('BOOTCAMP',), ('STUDENT_LOGS',)]

In [20]:
for tabla in lista_tablas:
    display(pd.read_sql_query(f"SELECT * FROM {tabla[0]}", conn))

,generated_id,Student_Name,Bootcamp_ID,Date_Start
0,1,Javi,11,20-11-2020
1,2,Javi,11,25-01-2021
2,3,Laura,32,20-11-2020
3,4,Martín,21,25-01-2021
4,5,Elena,11,25-01-2021
5,6,Clara,42,12-04-2021
6,7,Borja,21,12-04-2021


,generated_id,Bootcamp_ID,Program,Bootcamp_Name
0,1,11,Full Time,Data Science
1,2,21,Full Time,Full Stack
2,3,31,Full Time,UX/UI
3,4,41,Full Time,Ciberseguridad
4,5,12,Part Time,Data Science
5,6,22,Part Time,Full Stack
6,7,32,Part Time,UX/UI
7,8,42,Part Time,Ciberseguridad


,Student_ID,Date_Start
0,6,2021/05-18:59
1,7,2021/05-18:59


**¿En qué Bootcamp está Elena?**

In [30]:
query_review = """
SELECT b.Bootcamp_Name || " en " || b.Program
FROM STUDENTS s
JOIN BOOTCAMP b ON s.Bootcamp_ID == b.Bootcamp_ID
WHERE s.Student_Name == "Elena"
"""
cursor.execute(query_review)
print(f"Elena está en el Bootcamp de {cursor.fetchall()[0][0]}")

# https://www.sqlitetutorial.net/sqlite-string-functions/sqlite-concat/

Elena está en el Bootcamp de Data Science en Full Time


---------------------------------------------------------------------------------------------------------------

¿Y qué es más importante que hacer `conn.commit()`? 

¡Hacer `conn.close()`!

In [31]:
conn.close()

--------------------------------------------------------------------------------------------

### Referencias

- [w3schools](https://www.w3schools.com/sql/default.asp)  es tu lugar de referencia por excelencia.
- Busca en [datatofish](https://datatofish.com/python-tutorials/) la sección **Databases & SQL**
- Practica [String Functions](https://mode.com/sql-tutorial/sql-string-functions-for-cleaning/)

#### ¿Buscas **cursos/tutoriales** gratuitos?

- https://sqlbolt.com/
- https://www.kaggle.com/learn/intro-to-sql

#### ¿Un poco más avanzado?

- https://www.kaggle.com/learn/advanced-sql


#### La respuesta a la primera pregunta 

- https://medium.com/tableplus/how-to-pronounce-sql-properly-s-q-l-or-sequel-7203a5185676

### Bonus de cosa interesante

In [32]:
import dtale
from sklearn import datasets

df = datasets.load_wine()['data'] 
dtale.show(df)

2021-05-17 21:06:20,662 - INFO     - NumExpr defaulting to 8 threads.
